### Analyzing Correlation Between Funding Stage and Percentage of Employees Laid Off

The funding stage column of our dataset contains information on which round of funding the company is on. Since a lot of these companies are startups, we have information from many companies across many different rounds of funding. We'd like to investigate if there is a correlation between these funding stages and the percentage of employees laid off. This correlation will help us determine if the funding stage of a company can be used to indicate if a company is more likely to perform large-scale layoffs.

In [ ]:
fig = plt.figure(figsize=(12, 5))
sns.boxplot(x='Stage', y='Percentage', data=df, hue='Stage')
plt.xticks(df['Stage'].unique(), rotation=45)
plt.ylabel('Percentage Laid Off')
plt.show()

Looking at this plot, we can see that it appears as the companies goes through more rounds of funding, the distributions for their percentage of employees laid off tends to become smaller, and more centered between 0% - 20% of employees laid off. This is an interesting trend, so let's see if we can build a linear regressor to help identify this trend. Since the funding stage is a categorical column, we will first need to convert it into ordinal data. Ordinal data is a type of qualitative data where you can identify a clear scale between the different categories. Since funding stages happen in a sequence, and are in essence just a count of the amount of times a started has gotten funding, we can convert this to a numerical scale. To ensure it was created correctly, we will regenerate the same plot but with our numerized versions of the `Stage` column instead.

In [ ]:
series = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
stage_numerizer = dict(zip([f'Series {s}' for s in series], np.arange(1, len(series)+1, 1)))
stage_numerizer['Seed'] = 0
numerizer = defaultdict(lambda : np.nan)
numerizer.update(stage_numerizer)
clear_funding = df[['Percentage', 'Funding', 'Stage']]
clear_funding.loc[:, 'Stage_numerized'] = clear_funding['Stage'].apply(lambda i: numerizer[i])

fig = plt.figure(figsize=(12, 5))
sns.boxplot(x='Stage_numerized', y='Percentage', data=clear_funding, hue='Stage')
plt.xlabel('Stage Numerized')
plt.ylabel('Percentage Laid Off')
plt.legend().remove()
plt.show()

Looks like we created it correctly. Lets see if we can perform a linear regression now and find a correlation between Stage Numerized and 


$ H_0: \text{There is no satistically significant correlation between the Stage\_numerized and Percentage columns} $

$ H_1: \text{There is a satistically significant correlation between the Stage\_numerized and Percentage columns} $

For this test, we will be using a p-value of 0.01. We will use a smaller p-value because we converted the data numerically, and are omitting some of the other funding stages that did not fit into this ordinal categorization. A smaller p-value will help eliminate some of the bias introduced by performing these modifications.

In [ ]:
outcome, predictors = patsy.dmatrices('Percentage ~ Stage_numerized', clear_funding)
model = sm.OLS(outcome, predictors)
results = model.fit()
p_value = results.pvalues[1]
t_value = results.tvalues[1]
stages = np.sort(clear_funding['Stage_numerized'].unique())

fig = plt.figure(figsize=(12, 5))
sns.boxplot(x='Stage_numerized', y='Percentage', data=clear_funding, hue='Stage')

predicted_percentage = results.params[0] + results.params[1] * stages
sns.lineplot(x=stages, y=predicted_percentage, color='red', linewidth=2, label='Linear Regression')
plt.xlabel('Stage Numerized')
plt.ylabel('Percentage Laid Off')
plt.legend(loc='upper right', handles=[
    mpatches.Patch(color='red', label='Linear Regression'),
    mpatches.Patch(color='none', label=f'P_value={np.round(p_value, 4)}'),
    mpatches.Patch(color='none', label=f'P_value={np.round(t_value, 4)}'),
])
plt.title('Linearly Regressed Layoff Percentage using Stage Numerized')
plt.show()

We succesfully disproved the null hypothesis with a p-value of $0.0$, this suggests that there is a statistically significant correlation between the numerized stage and the percentage of employees laid off. With a t-value of $-11.5861$, we find that for every round of funding, a company that performs layoffs is expted to layoff around $10\%$ less employees. This means that if you were looking to identify a company to be more likely to perform a large-scale layoff, it would be a company that is in the lower funding stages.